In [6]:
import pandas as pd
import json

In [7]:
def get_position_name(position: int):
    match position:
        case 1:
            return "GKP"
        case 2:
            return "DEF"
        case 3:
            return "MID"
        case 4:
            return "FWD"

In [8]:
RAW_FILENAME = "raw/fpl_stats_26-08-2024.json"

In [24]:
with open(RAW_FILENAME,"r",encoding="utf-8") as f: # Source: https://fantasy.premierleague.com/api/bootstrap-static/
    all_data_raw = json.load(f)

In [25]:
all_data = pd.DataFrame(all_data_raw["elements"])

Filter to only the columns we need

In [26]:
allowed_cols = ["web_name","now_cost","ict_index","total_points","points_per_game","element_type","team_code","form"]
player_data = all_data[allowed_cols]

In [27]:
player_data["position"] = player_data["element_type"].apply(lambda x: get_position_name(x))
player_data = player_data.drop(columns=["element_type"])
player_data = player_data.rename(columns={"web_name":"name","now_cost":"cost"})

C:\Users\Crafty\AppData\Local\Temp\ipykernel_15196\2399959204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["position"] = player_data["element_type"].apply(lambda x: get_position_name(x))


In [28]:
player_data = player_data.loc[all_data["status"]=="a"]

In [29]:
player_data.head()

,name,cost,ict_index,total_points,points_per_game,team_code,form,position
0,Fábio Vieira,54,0.0,0,0.0,3,0.0,MID
2,Gabriel,60,4.6,12,6.0,3,6.0,DEF
3,Havertz,81,14.5,14,7.0,3,7.0,FWD
5,J.Timber,55,1.8,7,3.5,3,3.5,DEF
6,Jorginho,49,0.0,0,0.0,3,0.0,MID


In [30]:
team_data = pd.DataFrame(all_data_raw["teams"])

In [31]:
team_data = team_data[["code","short_name"]]

In [32]:
team_data

,code,short_name
0,3,ARS
1,7,AVL
2,91,BOU
3,94,BRE
4,36,BHA
5,8,CHE
6,31,CRY
7,11,EVE
8,54,FUL
9,40,IPS


In [33]:
def team_from_code(team_code):
    return team_data.loc[team_data["code"]==team_code]["short_name"].values[0]

In [34]:
player_data["team"] = player_data["team_code"].apply(lambda x: team_from_code(x))

In [35]:
player_data = player_data.drop(columns=["team_code"])

In [36]:
player_data.head()

,name,cost,ict_index,total_points,points_per_game,form,position,team
0,Fábio Vieira,54,0.0,0,0.0,0.0,MID,ARS
2,Gabriel,60,4.6,12,6.0,6.0,DEF,ARS
3,Havertz,81,14.5,14,7.0,7.0,FWD,ARS
5,J.Timber,55,1.8,7,3.5,3.5,DEF,ARS
6,Jorginho,49,0.0,0,0.0,0.0,MID,ARS


In [37]:
player_data.to_csv("./data/data.csv",index_label="id")